### Spark SQL and DataFrames

In [1]:
import org.apache.spark.sql.SparkSession 

val spark = SparkSession
 .builder
 .appName("SparkSQLExampleApp")
 .getOrCreate()

// Path to data set 
val csvFile="C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

// Read and create a temporary view
val df = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .load(csvFile)

// Create a temporary view
df.createOrReplaceTempView("us_delay_flights_tbl")


Intitializing Scala interpreter ...

Spark Web UI available at http://L2203100.bosonit.local:4041
SparkContext available as 'sc' (version = 3.0.3, master = local[*], app id = local-1651735049543)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3855a2cf
csvFile: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv
df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [2]:
spark.sql("""SELECT distance, origin, destination 
FROM us_delay_flights_tbl WHERE distance > 1000 
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [3]:
spark.sql("""SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [4]:
spark.sql("""SELECT delay, origin, destination,
 CASE
 WHEN delay > 360 THEN 'Very Long Delays'
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
 WHEN delay = 0 THEN 'No Delays'
 ELSE 'Early'
 END AS Flight_Delays
 FROM us_delay_flights_tbl
 ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [8]:
// Viewing the metadata

spark.catalog.listDatabases()
spark.catalog.listTables()
spark.catalog.listColumns("us_delay_flights_tbl")

res6: org.apache.spark.sql.Dataset[org.apache.spark.sql.catalog.Column] = [name: string, description: string ... 4 more fields]


In [9]:
// You can simply use SQL to query the table and assign the returned result to a DataFrame

val usFlightsDF = spark.sql("SELECT * FROM us_delay_flights_tbl")
val usFlightsDF2 = spark.table("us_delay_flights_tbl")

usFlightsDF: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]
usFlightsDF2: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


DataFrameReader is the core construct for reading data from a data source into a DataFrame.

DataFrameReader.format(args).option("key", "value").schema(args).load()

No schema is needed when reading from a static Parquet data source—the Parquet metadata usually contains the schema, so it’s inferred.

Parquet is the default and preferred data source for Spark because it’s efficient.

In [1]:
// Use Parquet

val file = """C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"""
val df = spark.read.format("parquet").load(file)

Intitializing Scala interpreter ...

Spark Web UI available at http://L2203100.bosonit.local:4041
SparkContext available as 'sc' (version = 3.0.3, master = local[*], app id = local-1651577530857)
SparkSession available as 'spark'


file: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [2]:
val df2 = spark.read.load(file)

df2: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [3]:
// Use CSV

val df3 = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .option("mode", "PERMISSIVE")
 .load("C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*")

df3: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [4]:
// Use JSON

val df4 = spark.read.format("json")
 .load("C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")

df4: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


DataFrameWriter does the reverse of its counterpart: it saves or writes data to a specified built-in data source. 

DataFrameWriter.format(args)
 .option(args)
 .bucketBy(args)
 .partitionBy(args)
 .save(path)
 
DataFrameWriter.format(args).option(args).sortBy(args).saveAsTable(table)


In [7]:
// Read Parquet files into a DataFrame

val file = """C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"""
val df = spark.read.format("parquet").load(file)

file: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


There’s no need to supply the schema, because Parquet saves it as part of its metadata.

In [11]:
// Reading Parquet les into a Spark SQL table

spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING parquet
 OPTIONS (
 path "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/" )""")

res8: org.apache.spark.sql.DataFrame = []


In [12]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [13]:
// Writing DataFrames to Parquet files

df.write.format("parquet")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/parquet/df_parquet")


In [14]:
// Writing DataFrames to Spark SQL tables

df.write
 .mode("overwrite")
 .saveAsTable("us_delay_flights_tbl")

In [8]:
val file = "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"
val df = spark.read.format("parquet").load(file)


file: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [10]:
// Read a JSON file into a DataFrame

val file = "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
val df = spark.read.format("json").load(file)

file: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [15]:
// Reading a JSON file into a Spark SQL table

spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING json
 OPTIONS (
 path "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")
""")

res12: org.apache.spark.sql.DataFrame = []


In [16]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [22]:
// Reading a CSV file into a DataFrame

val file = "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
val schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"

val df = spark.read.format("csv")
 .schema(schema)
 .option("header", "true")
 .option("mode", "FAILFAST") // Exit if any errors
 .option("nullValue", "") // Replace any null data with quotes
 .load(file)

file: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*
schema: String = DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [23]:
// Reading a CSV file into a Spark SQL table

spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING csv
 OPTIONS (
 path "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*",
 header "true",
 inferSchema "true",
 mode "FAILFAST"
 )""")

res16: org.apache.spark.sql.DataFrame = []


In [24]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [25]:
// Writing DataFrames to CSV files

df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

In [27]:
// Reading an ORC file into a DataFrame

val file = "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
val df = spark.read.format("orc").load(file)
df.show(10, false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



file: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [28]:
// Writing DataFrames to ORC files

df.write.format("orc")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/orc/df_orc")


In [29]:
// Reading an image file into a DataFrame

import org.apache.spark.ml.source.image
val imageDir = "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val imagesDF = spark.read.format("image").load(imageDir)
imagesDF.printSchema
imagesDF.select("image.height", "image.width", "image.nChannels", "image.mode",
 "label").show(5, false)


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)

+------+-----+---------+----+-----+
|height|width|nChannels|mode|label|
+------+-----+---------+----+-----+
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |1    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
+------+-----+---------+----+-----+
only showing top 5 rows



import org.apache.spark.ml.source.image
imageDir: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/
imagesDF: org.apache.spark.sql.DataFrame = [image: struct<origin: string, height: int ... 4 more fields>, label: int]


In [30]:
// Reading a binary file into a DataFrame

val path = "C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val binaryFilesDF = spark.read.format("binaryFile")
 .option("pathGlobFilter", "*.jpg")
 .load(path)
binaryFilesDF.show(5)

+--------------------+--------------------+------+--------------------+-----+
|                path|    modificationTime|length|             content|label|
+--------------------+--------------------+------+--------------------+-----+
|file:/C:/Users/al...|2022-04-26 18:34:...| 55037|[FF D8 FF E0 00 1...|    0|
|file:/C:/Users/al...|2022-04-26 18:34:...| 54634|[FF D8 FF E0 00 1...|    1|
|file:/C:/Users/al...|2022-04-26 18:34:...| 54624|[FF D8 FF E0 00 1...|    0|
|file:/C:/Users/al...|2022-04-26 18:34:...| 54505|[FF D8 FF E0 00 1...|    0|
|file:/C:/Users/al...|2022-04-26 18:34:...| 54475|[FF D8 FF E0 00 1...|    0|
+--------------------+--------------------+------+--------------------+-----+
only showing top 5 rows



path: String = C:/Users/alice.marchi/Downloads/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/
binaryFilesDF: org.apache.spark.sql.DataFrame = [path: string, modificationTime: timestamp ... 3 more fields]


In [31]:
val file = """C:/tmp/output/fire_parquet"""
val df = spark.read.format("parquet").load(file)

file: String = C:/tmp/output/fire_parquet
df: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [33]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW fire_tbl
 USING parquet
 OPTIONS (
 path "C:/tmp/output/fire_parquet" )
""")

res25: org.apache.spark.sql.DataFrame = []


In [35]:
spark.sql("SELECT * FROM fire_tbl").show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|      Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+--

In [36]:
val file = "C:/tmp/output/fire_json"
val df = spark.read.format("json").load(file)
df.show(5)

+-------+--------------------+--------------------+---------+----+----------+--------------------+----------+----------------+-------------+----+---------+-------------+----------------------+--------------+--------------------+--------------------+---------+----------------+--------+-------------+-----------+------------------+------+--------------------------+--------+----------+-------+
|ALSUnit|             Address|       AvailableDtTm|Battalion| Box|  CallDate|CallFinalDisposition|CallNumber|        CallType|CallTypeGroup|City|    Delay|FinalPriority|FirePreventionDistrict|IncidentNumber|            Location|        Neighborhood|NumAlarms|OriginalPriority|Priority|        RowID|StationArea|SupervisorDistrict|UnitID|UnitSequenceInCallDispatch|UnitType| WatchDate|Zipcode|
+-------+--------------------+--------------------+---------+----+----------+--------------------+----------+----------------+-------------+----+---------+-------------+----------------------+--------------+-------

file: String = C:/tmp/output/fire_json
df: org.apache.spark.sql.DataFrame = [ALSUnit: boolean, Address: string ... 26 more fields]
